### Importamos los datos en un dataframe

In [2]:
import pandas as pd
df = pd.read_csv('insurance.csv')

### Breve Análisis del dataset

Mostramos primero algunos valores de nuestro dataset.

Luego los nombres de las columnas de nuestro dataframe

Y para finalizar tambien mostramos el tamaño del dataframe con el que trabajaremos

In [3]:
print("Next the first rows of our dataset are showed: \n")
print(df.head())
print("\nThe colums of our dataset refears to: \n"+str(df.columns) + "\n")
print("the size of the dataframe is: \n " + str(df.shape) + "\n")

Next the first rows of our dataset are showed: 

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520

The colums of our dataset refears to: 
Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

the size of the dataframe is: 
 (1338, 7)



Con la siguiente linea nos fijamos los tipos de datos que tenemos presentes en cada columna

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


Verificamos la cantidad de datos nulos presentes en nuestro dataframe.

In [5]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

Podemos utilizar el método describe para examinar como son los valores que se presentan dentro del dataframe.

In [6]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


### Breve análisis del dataframe

De ver los valores que devuelve describe podemos darnos cuenta que como los valores minimos de edad no son 0 entonces no hay datos nulos ni ceros que podrían se considerados invalidos en esa columna, de hecho el valor maximo es 64 por lo que podríamos plantear a priori que no es necesario una limpieza de los datos para esa columna.
Luego en el BMI vemos que no existen valores iguales a 0 de hecho el valor minimo es 15 lo cual es un valor posible y el valor maximo es 53 lo cual es un valor muy alto pero al ser el valor mas alto de nuestro dataframe podríamos considerar que esta "bien" y que luego será el diagrama de caja el que nos termine de definir si pertenece o no al grupo de los outlayers, pero en principio uno podría afirmar que el valor es posible en un humano, muy poco saludable pero posible. 
La cantidad de hijos tambien es un parámetro que no presenta valores nulos ni invalidos, debido a que el minimo es cero el cual es un valor perfectamente posible y el valor maximo es 5 lo cual tambien parecería ser un valor esperado, por lo tanto no es necesario realizar ningun tipo de limpieza de datos sobre el mismo.
Los charges que son sobrados por la atencion medica recibida en principio tampoco tienen valores nulos ni ceros lo cual implica que no hay que realizar una limpieza de los datos, pero es de notar que la varianza es elevada, el valor minimo es cercano a 1000 y el valor maximo es cercano a 60.000 lo cual nos invita a pensar en como se podrían limpiar estas métricas, no porque las mismas estén mal tipeadas, sino mas bien porque es probable que nuestro modelo no requiera saber la informacion de atenciones medicas tan bajas de costo o tan altas, lo decidirá en parte el diagrama de caja y lo bien que trabae nuestro modelo.

### Datos que faltan

Existen ciertas columnas que el metodo describe no es capaz de computar debido a que las mismas son columnas que se encuentran descriptas por strings. Para manejar estos datos lo que haremos será ver como estan compuestos los mismos, para eso podemos observar que en la columna "smoker" lo que se indica es si la persona es fumadora o no, esto se indica mediante las palabras "yes" o "no". Por lo tanto lo que podemos hacer es reemplazar todas las apariciones de yes por un 1 y todas las operaciones de no por un 0, esto lo podemos hacer de esta manera ya que es una variable categórica que presenta solo 2 estados posibles por lo que al asignarle un numero a cada una no le añadimos un peso consigo, no implica un ordenamiento de las categorías.
En cambio para el caso de los barrios ahí tenemos otro problema distinto debido a que ahora es una distribucion categótica pero con varias categorías, por lo que no podemos proceder de la misma manera que antes.

A continuacion se muestra una manera de saber los posibles valores que pueden tomar cada una de estas columnas.


In [7]:
# count occurances a particular column
occur = df.groupby(['smoker']).size()
  
# diplay occurances of a particular column
display(occur)

smoker
no     1064
yes     274
dtype: int64

In [8]:
# count occurances a particular column
occur = df.groupby(['region']).size()
  
# diplay occurances of a particular column
display(occur)

region
northeast    324
northwest    325
southeast    364
southwest    325
dtype: int64

Primero resolvamos el problema de la columna smokers: 

Para esto lo que tenemos que hacer es reemplazar todos los yes por 1 y todos los no por 0:

In [10]:
df[['smoker']] = df[['smoker']].replace('yes',1)
df[['smoker']] = df[['smoker']].replace('no',0)

Para comprobar que haya funcionado como esperamos podemos comprobar a continuacion que describe ahora sí contempla la columna smoker y que la misma presenta una gran cantidad de no (0) y poca cantidad de yes (1) (la proporcion esta dada por la media que es aproximadamente 0.205), por lo tanto solo 1/5 de las personas son fumador

In [11]:
df.describe()

,age,bmi,children,smoker,charges
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,0.204783,13270.422265
std,14.049960,6.098187,1.205493,0.403694,12110.011237
min,18.000000,15.960000,0.000000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,0.000000,9382.033000
75%,51.000000,34.693750,2.000000,0.000000,16639.912515
max,64.000000,53.130000,5.000000,1.000000,63770.428010


Para el análisis de la variable en la columna region, como el numero de categorías es pequeño (4), podemos plantear una clasificacion mediante One Hot Encoding en lugar de utilizar embebbing, ya que de esta manera el análisis es simple y la cantidad de variables que agregamos no es tan grande como para necesitar realizar el cambio a embebbing.

A continuación se detalla el procedimiento por el cual se realiza la codificacion mediante One Hot Encoding:

In [12]:
# Get one hot encoding of columns Bs
one_hot = pd.get_dummies(df['region'])
# Drop column B as it is now encoded
df = df.drop('region',axis = 1)
# Join the encoded df
df = df.join(one_hot)

A continuación mostramos como quedó el dataset:

In [13]:
df.head()

,age,sex,bmi,children,smoker,charges,northeast,northwest,southeast,southwest
0,19,female,27.900,0,1,16884.92400,0,0,0,1
1,18,male,33.770,1,0,1725.55230,0,0,1,0
2,28,male,33.000,3,0,4449.46200,0,0,1,0
3,33,male,22.705,0,0,21984.47061,0,1,0,0
4,32,male,28.880,0,0,3866.85520,0,1,0,0


Simplemente por una cuestion de orden pondremos la variable que no interesa hallar como la ultima columna del dataframe:

In [14]:
df = df[['age', 'sex', 'bmi', 'children', 'smoker', 'northeast', 'northwest', 'southeast', 'southwest', 'charges']]

In [15]:
df.head()

,age,sex,bmi,children,smoker,northeast,northwest,southeast,southwest,charges
0,19,female,27.900,0,1,0,0,0,1,16884.92400
1,18,male,33.770,1,0,0,0,1,0,1725.55230
2,28,male,33.000,3,0,0,0,1,0,4449.46200
3,33,male,22.705,0,0,0,1,0,0,21984.47061
4,32,male,28.880,0,0,0,1,0,0,3866.85520


De esta manera hemos finalizado la etapa de limpieza y ordenamiento de los datos que nos permitirá a partir de ahora trabaja de manera mas estructurada con los mismos.